In [7]:
from enum import Enum

import schema as s
from experta import *
from experta.fact import *

In [6]:
class Actions(Enum):
    MAKE_COFFEE = "make coffee"
    CONSERVE = "conserve"
    STOP = "stop"


actions = [Actions.MAKE_COFFEE, Actions.CONSERVE, Actions.STOP]


class CoffeeMachine(Fact):
    """A fact representing the coffee machine."""
    pass


class Coffee(Fact):
    needs_milk = Field(bool, mandatory=True)
    water_use = Field((s.Use(float), lambda l: 0.0 <= l <= 1.0), default=1.0)


class Espresso(Coffee):
    needs_milk = False


class Latte(Coffee):
    needs_milk = True


coffees = [Latte, Espresso]


class Component(Fact):
    """A fact representing a component of the coffee machine."""
    pass


class Water(Component):
    """A fact representing the water component."""
    level = Field(s.And(s.Use(float), lambda l: 0.0 <= l <= 1.0), default=1.0)
    threshold = 0.2
    # This should be set at the beginning
    water_hardness = Field(s.Or("soft", "medium", "hard"))


class Milk(Component):
    """A fact representing the milk component."""
    level = Field(s.And(s.Use(float), lambda l: 0.0 <= l <= 1.0), default=1.0)
    # Add state here too?


class Grinder(Component):
    """A fact representing the grinder component."""
    filled = Field(bool, default=False)
    state = Field(s.Or("grinding", "inactive", "glitch"), default="inactive")
    degree = Field(s.And(s.Use(int), lambda x: 1 <= x <=5), default=1)


class Heater(Component):
    """A fact representing the heater component."""
    temperature = Field(s.And(s.Use(float)))


class Brew(Fact):
    """A fact representing the brewing process."""
    pass


class Nozzle(Component):
    """A fact representing the nozzle for dispensing coffee."""
    status = Field(s.Or("blocked", "nozzling", "inactive"))


def get_user_coffee():
    print("Choose coffee")
    for i, coffee in enumerate(coffees):
        print(f"[{i}] {coffee.__name__}")
    cof = int(input("Choice: "))
    return coffees[cof] if 0 <= cof < len(coffees) else None


def get_user_input():
    print("Choose from a set of possible actions")
    for i, action in enumerate(actions):
        print(f"[{i}] {action.value}")
    act = int(input("Choice: "))
    return actions[act] if 0 <= act < len(actions) else Actions.STOP


class UserInput(Fact):
    """A fact representing user input for the coffee machine."""
    # user can choose coffee, hot water or ex. system conservation
    action = Field(s.Or(**actions))

TypeError: schema.Or() argument after ** must be a mapping, not list

In [ ]:
class CoffeeMachineRules(KnowledgeEngine):
    """The rule-based system for the coffee machine."""

    @Rule(CoffeeMachine())
    def startup(self):
        print("Coffee machine started.")

    @Rule(AND(CoffeeMachine(), Component()))
    def check_component(self):
        print("Checking component:", self.fact)

    @Rule(AND(CoffeeMachine(), Water(level=P(lambda x: x <= 0))))
    def add_water(self):
        print("Add water to the coffee machine.")

    @Rule(AND(CoffeeMachine(), CoffeeBeans(amount=P(lambda x: x <= 0))))
    def add_coffee(self):
        print("Add coffee to the coffee machine.")

    @Rule(AND(CoffeeMachine(), Milk(amount=P(lambda x: x <= 0))))
    def add_milk(self):
        print("Add milk to the coffee machine.")

    @Rule(AND(CoffeeMachine(), Grinder(status="empty")))
    def fill_grinder(self):
        print("Fill grinder with coffee beans.")

    @Rule(AND(CoffeeMachine(), Heater(status="off")))
    def turn_on_heater(self):
        print("Turn on heater.")

    @Rule(AND(CoffeeMachine(), Brew(status="ready")))
    def start_brewing(self):
        print("Start brewing.")

    @Rule(AND(CoffeeMachine(), Nozzle(status="blocked")))
    def clear_nozzle(self):
        print("Clear nozzle for dispensing.")

    @Rule(AND(CoffeeMachine(), UserInput(action="make_coffee")))
    def make_coffee(self):
        print("Coffee brewing process initiated.")

In [ ]:
# Creating a coffee machine instance
coffee_machine = CoffeeMachine()

# Initializing the rules engine
engine = CoffeeMachineRules()
engine.reset()
engine.declare(coffee_machine)

# Example usage: Simulate user making coffee
engine.declare(UserInput(action="make_coffee"))
engine.run()